In [78]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import gradio as gr


In [79]:
# Null 처리 함수
def fillna(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    df['Fare'] = df['Fare'].fillna(0)
    return df

In [80]:
# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

In [81]:
# 레이블 인코딩 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

In [82]:
# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [83]:
titanic_df = pd.read_csv('../Ch01/titanic_train.csv')
y_titanic_df = titanic_df['Survived'] # 생존 여부 예측을 위한 label
X_titanic_df = titanic_df.drop('Survived', axis=1)

X_titanic_df = transform_features(X_titanic_df)

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

In [85]:
# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear') # liblinear가 작은 데이터 이진 분류에는 적합

In [86]:
# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifire 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))

DecisionTreeClassifier 정확도: 0.7877
RandomForestClassifire 정확도: 0.8547
LogisticRegression 정확도: 0.8659


In [87]:
# 하이퍼파라미터 서치 및 교차 검증
parameters = {'max_depth':[2,3,5,10], 
              'min_samples_split':[2,3,5], # 노드를 분할하기 위한 최소 샘플 수
              'min_samples_leaf':[1,5,8]} # 리프 노드에 포함되어야 하는 최소 샘플 수
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터:', grid_dclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.7992


In [88]:
# GridSearch의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행
# refit=True이므로 재학습 필요 없음
dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print('테스트 세트에서의 DecisionTreeClassifier 정확도 : {0:.4f}'.format(accuracy))

테스트 세트에서의 DecisionTreeClassifier 정확도 : 0.8715


In [89]:
# 피처 중요도를 확인 
importance = best_dclf.feature_importances_
importance = np.argsort(importance)[::-1]
print(X_titanic_df.columns[importance])

Index(['Sex', 'Pclass', 'Age', 'Cabin', 'Fare', 'Embarked', 'Parch', 'SibSp'], dtype='object')


In [90]:
# 데이터 선택
features = ['Sex', 'Pclass', 'Age']
X = X_titanic_df[features]
y = y_titanic_df

In [95]:
# 하이퍼파라미터 서치 및 교차 검증
parameters = {'max_depth':[2,3,5,10], 
              'min_samples_split':[2,3,5], # 노드를 분할하기 위한 최소 샘플 수
              'min_samples_leaf':[1,5,8]} # 리프 노드에 포함되어야 하는 최소 샘플 수
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X, y)

print('GridSearchCV 최적 하이퍼 파라미터:', grid_dclf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.7958


In [92]:
def predict_survival(Sex, Pclass, Age):
    Sex = 0 if Sex == "male" else 1
    input_data = pd.DataFrame([[Sex, Pclass, Age]], columns=["Sex","Pclass","Age"])
    prediction = best_dclf.predict(input_data)
    return "Survived" if prediction[0] == 1 else "Did not survive"

interface = gr.Interface(
    fn=predict_survival,
    inputs = [
        gr.Radio(choices=["male", "female"], label="Sex"),
        gr.Radio(choices=[1,2,3], label="Passenger Class"),
        gr.Slider(minimum=0, maximum=80, value=30, label="Age")
    ],
    outputs = "text"
).launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
